<a href="https://colab.research.google.com/github/HunterRSloan/Heart-Disease-Predictor/blob/main/SEProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving BlockchainProgAssSc1.PNG to BlockchainProgAssSc1.PNG


In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier

# Load the dataset
df = pd.read_csv("heart.csv")

# Split the dataset into features (X) and target (y)
X = df[["age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", "thalach", "exang", "oldpeak", "slope", "ca", "thal"]]
y = df["target"]

# Calculate the midpoint index of the data
midpoint = len(df) // 2

# Split the data into training and testing sets
X_train = X.iloc[:midpoint, :]
y_train = y.iloc[:midpoint]
X_test = X.iloc[midpoint:, :]
y_test = y.iloc[midpoint:]

# Create a Random Forest classifier
rf_model = RandomForestClassifier(random_state=42)

# Train the Random Forest model
rf_model.fit(X_train, y_train)

specified_row_index = 14

# Extract the specified row of data for prediction (excluding the "target" column)
specified_row_features = X.iloc[specified_row_index:specified_row_index + 1, :]

# Predict the likelihood of heart disease for the specified row
predicted_likelihood = rf_model.predict_proba(specified_row_features)[:, 1]

# Print the predicted likelihood
print("Predicted Likelihood of Heart Disease:", predicted_likelihood[0])

# Evaluate the model on the testing data
test_accuracy = rf_model.score(X_test, y_test)
print("Testing Accuracy:", test_accuracy)
print(df)

Predicted Likelihood of Heart Disease: 0.16
Testing Accuracy: 0.9473684210526315
      age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  \
0      52    1   0       125   212    0        1      168      0      1.0   
1      53    1   0       140   203    1        0      155      1      3.1   
2      70    1   0       145   174    0        1      125      1      2.6   
3      61    1   0       148   203    0        1      161      0      0.0   
4      62    0   0       138   294    1        1      106      0      1.9   
...   ...  ...  ..       ...   ...  ...      ...      ...    ...      ...   
1020   59    1   1       140   221    0        1      164      1      0.0   
1021   60    1   0       125   258    0        0      141      1      2.8   
1022   47    1   0       110   275    0        0      118      1      1.0   
1023   50    0   0       110   254    0        0      159      0      0.0   
1024   54    1   0       120   188    0        1      113      0      1.

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, matthews_corrcoef
import numpy as np

# Load the dataset
df = pd.read_csv("heart.csv")

# Split the dataset into features (X) and target (y)
X = df[["age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", "thalach", "exang", "oldpeak", "slope", "ca", "thal"]]
y = df["target"]

# Create a Random Forest classifier
rf_model = RandomForestClassifier(random_state=42)

# Train the Random Forest model
rf_model.fit(X, y)

# Get feature importances from the trained model
feature_importances = rf_model.feature_importances_

# Define a threshold for selecting the least important features
threshold = 0.05  # Adjust this threshold as needed

# Filter features based on the threshold
selected_features = X.columns[feature_importances >= threshold]

# Find the removed features (those below the threshold)
removed_features = X.columns[feature_importances < threshold]

# Update the feature matrix (X) to include only the selected features
X = X[selected_features]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the Random Forest model on the updated feature matrix
rf_model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = rf_model.predict(X_test)

# Calculate the accuracy of the model
new_accuracy = accuracy_score(y_test, y_pred)

# Calculate confusion matrix
rf_cm = confusion_matrix(y_test, y_pred)

# Sensitivity, Specificity, and MCC functions
def sensitivity_specificity_mcc(cm):
    tn, fp, fn, tp = cm.ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    mcc = matthews_corrcoef(y_test, y_pred)
    return sensitivity, specificity, mcc

# Calculate metrics
rf_sensitivity, rf_specificity, rf_mcc = sensitivity_specificity_mcc(rf_cm)

# Print Feature Importances
print("Feature Importances:", feature_importances, "\n")

# Print Importance Threshold
print("Threshold:", threshold, "\n")

# Print Removed Features
print("Removed Features:")
print(removed_features, "\n")

# Print the new accuracy
print("New Testing Accuracy:", new_accuracy, "\n")

# Print Sensitivity, Specificity, and MCC
print("Sensitivity:", rf_sensitivity)
print("Specificity:", rf_specificity)
print("MCC:", rf_mcc, "\n")

# Define the number of folds for cross-validation
num_folds = 5
cv_scores = cross_val_score(rf_model, X, y, cv=num_folds, scoring='accuracy')

# Print the cross-validation results
print("Cross-Validation Accuracies:", cv_scores)
print("Mean Accuracy:", np.mean(cv_scores))
print("Standard Deviation:", np.std(cv_scores))


Feature Importances: [0.08931303 0.03605681 0.13420131 0.0742534  0.07892956 0.00887372
 0.01961871 0.12047307 0.05959222 0.11615126 0.04873837 0.11675515
 0.0970434 ] 

Threshold: 0.05 

Removed Features:
Index(['sex', 'fbs', 'restecg', 'slope'], dtype='object') 

New Testing Accuracy: 0.9853658536585366 

Sensitivity: 0.970873786407767
Specificity: 1.0
MCC: 0.9711511393019859 

Cross-Validation Accuracies: [1.         1.         0.98536585 0.98536585 0.98536585]
Mean Accuracy: 0.9912195121951219
Standard Deviation: 0.007169238271560528


In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

# Load the dataset
df = pd.read_csv("heart.csv")

# Split the dataset into features (X) and target (y)
X = df[["age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", "thalach", "exang", "oldpeak", "slope", "ca", "thal"]]
y = df["target"]

# Create a Random Forest classifier
rf_model = RandomForestClassifier(random_state=42)

# Define the number of folds for cross-validation
num_folds = 5

# Create StratifiedKFold object
stratified_kf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

# Initialize a list to store cross-validation accuracies
cv_accuracies = []

# Perform cross-validation
for train_index, test_index in stratified_kf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Train the Random Forest model on the training data
    rf_model.fit(X_train, y_train)

    # Make predictions on the testing data
    y_pred = rf_model.predict(X_test)

    # Calculate the accuracy of the model
    accuracy = accuracy_score(y_test, y_pred)

    # Append the accuracy to the list
    cv_accuracies.append(accuracy)

# Calculate the mean and standard deviation of cross-validation accuracies
mean_accuracy = sum(cv_accuracies) / num_folds
std_deviation = np.std(cv_accuracies)

# Print the cross-validation results
print("Cross-Validation Accuracies:", cv_accuracies)
print("Mean Accuracy:", mean_accuracy)
print("Standard Deviation:", std_deviation)


Cross-Validation Accuracies: [1.0, 1.0, 1.0, 0.9804878048780488, 1.0]
Mean Accuracy: 0.9960975609756098
Standard Deviation: 0.007804878048780495


In [ ]:
#Neural Networks Model

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, matthews_corrcoef

# Load the dataset
df = pd.read_csv("heart.csv")

# Split the dataset into features (X) and target (y)
X = df[["age", "sex", "cp", "trestbps", "chol", "restecg", "thalach", "exang", "oldpeak", "slope", "ca", "thal"]]
y = df["target"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the input features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the neural network model with L2 regularization
model = keras.Sequential([
    keras.layers.Input(shape=(X_train.shape[1],), name='input_layer'),
    keras.layers.Dense(64, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dense(32, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dense(1, activation='sigmoid')
])

# Define the custom learning rate
custom_learning_rate = 0.001

# Create an optimizer with the custom learning rate
custom_optimizer = keras.optimizers.Adam(learning_rate=custom_learning_rate)

# Compile the model with the custom optimizer
model.compile(optimizer=custom_optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Implement Early Stopping Callback
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=20,
    restore_best_weights=True
)

# Train the model with early stopping
history = model.fit(X_train, y_train, epochs=100, batch_size=32,
                    validation_data=(X_test, y_test), callbacks=[early_stopping])

# Calculate the likelihood of a specific row having heart disease (replace X_new with your data)
X_new = scaler.transform(X_test)  # Standardize the new data
likelihood = model.predict(X_new)
with np.printoptions(precision=15, suppress=True, threshold=np.inf):
    print("Likelihood of heart disease for the entire test set:")
    print(likelihood)

# Cross-validation (you can include L2 regularization here as well)
# Define the number of folds for cross-validation
num_folds = 5

# Create StratifiedKFold object
stratified_kf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

# Initialize a list to store cross-validation accuracies
cv_accuracies = []
cv_sensitivities = []
cv_specificities = []
cv_mccs = []

# Perform cross-validation
for train_index, test_index in stratified_kf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Define the neural network model with L2 regularization
    model = keras.Sequential([
        keras.layers.Input(shape=(X_train.shape[1],), name='input_layer'),
        keras.layers.Dense(64, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)),
        keras.layers.Dense(32, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)),
        keras.layers.Dense(1, activation='sigmoid')
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Train the model on the training data
    model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

    # Calculate the likelihood of heart disease for the test set
    test_likelihood = model.predict(scaler.transform(X_test))

    # Round likelihoods to binary predictions
    y_pred = (test_likelihood > 0.5).astype(int)

    # Calculate confusion matrix
    nn_cm = confusion_matrix(y_test, y_pred)

    # Sensitivity, Specificity, and MCC functions
    def sensitivity_specificity_mcc(cm):
        tn, fp, fn, tp = cm.ravel()
        sensitivity = tp / (tp + fn)
        specificity = tn / (tn + fp)
        mcc = matthews_corrcoef(y_test, y_pred)
        return sensitivity, specificity, mcc

    # Calculate metrics
    nn_sensitivity, nn_specificity, nn_mcc = sensitivity_specificity_mcc(nn_cm)

    # Append the accuracy and metrics to the respective lists
    cv_accuracies.append(accuracy_score(y_test, y_pred))
    cv_sensitivities.append(nn_sensitivity)
    cv_specificities.append(nn_specificity)
    cv_mccs.append(nn_mcc)

# Calculate the mean and standard deviation of cross-validation accuracies, sensitivities, specificities, and MCCs
mean_accuracy = sum(cv_accuracies) / num_folds
std_deviation_accuracy = np.std(cv_accuracies)
mean_sensitivity = sum(cv_sensitivities) / num_folds
std_deviation_sensitivity = np.std(cv_sensitivities)
mean_specificity = sum(cv_specificities) / num_folds
std_deviation_specificity = np.std(cv_specificities)
mean_mcc = sum(cv_mccs) / num_folds
std_deviation_mcc = np.std(cv_mccs)

# Print the cross-validation results
print("Cross-Validation Accuracies:", cv_accuracies)
print("Mean Accuracy:", mean_accuracy)
print("Standard Deviation (Accuracy):", std_deviation_accuracy)
print("Mean Sensitivity:", mean_sensitivity)
print("Standard Deviation (Sensitivity):", std_deviation_sensitivity)
print("Mean Specificity:", mean_specificity)
print("Standard Deviation (Specificity):", std_deviation_specificity)
print("Mean MCC:", mean_mcc)
print("Standard Deviation (MCC):", std_deviation_mcc)


Epoch 1/100
26/26 [==============================] - 2s 17ms/step - loss: 1.1236 - accuracy: 0.8049 - val_loss: 1.0361 - val_accuracy: 0.7805
Epoch 2/100
26/26 [==============================] - 0s 12ms/step - loss: 0.9232 - accuracy: 0.8524 - val_loss: 0.9267 - val_accuracy: 0.8000
Epoch 3/100
26/26 [==============================] - 0s 14ms/step - loss: 0.8107 - accuracy: 0.8610 - val_loss: 0.8491 - val_accuracy: 0.8098
Epoch 4/100
26/26 [==============================] - 0s 7ms/step - loss: 0.7276 - accuracy: 0.8695 - val_loss: 0.7858 - val_accuracy: 0.8000
Epoch 5/100
26/26 [==============================] - 0s 5ms/step - loss: 0.6628 - accuracy: 0.8768 - val_loss: 0.7309 - val_accuracy: 0.8244
Epoch 6/100
26/26 [==============================] - 0s 5ms/step - loss: 0.6088 - accuracy: 0.8878 - val_loss: 0.6824 - val_accuracy: 0.8244
Epoch 7/100
26/26 [==============================] - 0s 6ms/step - loss: 0.5665 - accuracy: 0.9037 - val_loss: 0.6467 - val_accuracy: 0.8146
Epoch 8/10

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Likelihood of heart disease for the entire test set:
[[0.9971863     ]
 [0.99999297    ]
 [0.8742965     ]
 [0.9984748     ]
 [0.8255358     ]
 [0.9999955     ]
 [0.88063127    ]
 [0.006030592   ]
 [0.0009281417  ]
 [0.017396048   ]
 [0.99280554    ]
 [0.964503      ]
 [0.48469865    ]
 [0.0032727383  ]
 [0.98785084    ]
 [0.9998548     ]
 [0.0006784439  ]
 [0.52224153    ]
 [0.13365954    ]
 [0.001284534   ]
 [0.99970204    ]
 [0.09776643    ]
 [0.9974397     ]
 [0.00025385537 ]
 [0.99970204    ]
 [0.00012885946 ]
 [0.9997305     ]
 [0.00011911215 ]
 [0.0006029664  ]
 [0.9999886     ]
 [0.0012651893  ]
 [0.9999227     ]
 [0.0303747     ]
 [0.99999297    ]
 [0.39668688    ]
 [0.9934439     ]
 [0.99970204    ]
 [0.9976819     ]
 [0.95905143    ]
 [0.50066936    ]
 [0.006030592   ]
 [0.99919635    ]
 [0.027828114   ]
 [0.9974397     ]
 [0.97789884    ]
 [0.0011312127  ]
 [0.000049828403]
 [0.0009829135  ]
 [0.036548927   ]
 [0.9713179     ]
 [0.0031587789  ]
 [0.0034601465  ]
 [0.7089346

In [ ]:
#Gradient Boosting / XGBoost

import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, recall_score, precision_score, matthews_corrcoef
import xgboost as xgb
import numpy as np

# Load the dataset
df = pd.read_csv("heart.csv")

# Split the dataset into features (X) and target (y)
X = df.drop('target', axis=1)
y = df["target"]
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the XGBoost classifier
model = xgb.XGBClassifier(objective='binary:logistic', random_state=42)

# Perform cross-validation
cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')

# Print the cross-validation scores
print("Cross-Validation Scores:", cv_scores)
print("Mean Accuracy:", np.mean(cv_scores))
print("Standard Deviation of Mean Accuracy:", np.std(cv_scores))

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate confusion matrix
xgb_cm = confusion_matrix(y_test, y_pred)

# Sensitivity, Specificity, and MCC functions
def sensitivity_specificity_mcc(cm):
    tn, fp, fn, tp = cm.ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    mcc = matthews_corrcoef(y_test, y_pred)
    return sensitivity, specificity, mcc

# Calculate metrics
xgb_sensitivity, xgb_specificity, xgb_mcc = sensitivity_specificity_mcc(xgb_cm)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Print the metrics
print(f'Accuracy: {accuracy * 100:.2f}%')
print(f'Precision: {precision * 100:.2f}%')
print(f'Recall: {recall * 100:.2f}%')
print(f'F1 Score: {f1 * 100:.2f}%')
print('Confusion Matrix:')
print(xgb_cm)
print("Sensitivity:", xgb_sensitivity)
print("Specificity:", xgb_specificity)
print("MCC:", xgb_mcc)


Cross-Validation Scores: [1.         0.96341463 1.         0.98170732 0.95731707]
Mean Accuracy: 0.9804878048780488
Standard Deviation of Mean Accuracy: 0.01783992541259489
Accuracy: 98.54%
Precision: 100.00%
Recall: 97.09%
F1 Score: 98.52%
Confusion Matrix:
[[102   0]
 [  3 100]]
Sensitivity: 0.970873786407767
Specificity: 1.0
MCC: 0.9711511393019859


In [ ]:
#Logistic Regression Model

import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, recall_score, precision_score, matthews_corrcoef
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np

# Load the dataset
df = pd.read_csv("heart.csv")

# Assuming your target variable is 'target' and you want to drop certain columns
X = df.drop('target', axis=1)
y = df["target"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Logistic Regression model
model = LogisticRegression(random_state=42, max_iter=1000)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Perform cross-validation
cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='accuracy')

# Print the cross-validation scores
print("Cross-Validation Scores:", cv_scores)
print("Mean Accuracy:", cv_scores.mean())
print("Standard Deviation of Mean Accuracy:", np.std(cv_scores))

# Train the Logistic Regression model
model.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test_scaled)

# Calculate confusion matrix
lr_cm = confusion_matrix(y_test, y_pred)

# Sensitivity, Specificity, and MCC functions
def sensitivity_specificity_mcc(cm):
    tn, fp, fn, tp = cm.ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    mcc = matthews_corrcoef(y_test, y_pred)
    return sensitivity, specificity, mcc

# Calculate metrics
lr_sensitivity, lr_specificity, lr_mcc = sensitivity_specificity_mcc(lr_cm)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Print the metrics
print(f'Accuracy: {accuracy * 100:.2f}%')
print(f'Precision: {precision * 100:.2f}%')
print(f'Recall: {recall * 100:.2f}%')
print(f'F1 Score: {f1 * 100:.2f}%')
print('Confusion Matrix:')
print(lr_cm)
print("Sensitivity:", lr_sensitivity)
print("Specificity:", lr_specificity)
print("MCC:", lr_mcc)


Cross-Validation Scores: [0.83536585 0.85365854 0.8902439  0.86585366 0.79878049]
Mean Accuracy: 0.8487804878048781
Standard Deviation of Mean Accuracy: 0.03070653246832724
Accuracy: 79.51%
Precision: 75.63%
Recall: 87.38%
F1 Score: 81.08%
Confusion Matrix:
[[73 29]
 [13 90]]
Sensitivity: 0.8737864077669902
Specificity: 0.7156862745098039
MCC: 0.5972548367142131
